#  Label arXiv with LAD data

We will use this in our subnational analysis of DL research

In [ ]:
%run notebook_preamble.ipy

In [ ]:
import geopandas as gp

from shapely.geometry import Point

## Load data

In [ ]:
grid = pd.read_csv('../data/external/grid/grid.csv')
grid_lat_lon = pd.read_csv('../data/external/grid/full_tables/addresses.csv')

### This is to fix a bug in the fuzzy matching - we matched UCL papers with UCL Australia instead of UK

In [ ]:
ucl = grid.loc[grid['Name']=='University College London'].reset_index(drop=True)

ucl.columns = ['institute_id','institute_name','institute_city','institute_state','institute_country']

In [ ]:
ucl_address = grid_lat_lon.set_index('grid_id').loc[ucl['institute_id'][0]]
ucl['institute_lat'] = ucl_address['lat']
ucl['institute_lon'] = ucl_address['lng']


In [ ]:
grid_matched = pd.read_csv('../data/external/1_8_2019_paper_institute_locations.csv',compression='zip',dtype={'article_id':str})

In [ ]:
grid_matched_w_data = grid_matched.dropna(axis=0,subset=['institute_lat','institute_lon'])

In [ ]:
ucl_papers = grid_matched_w_data.loc[grid_matched_w_data['institute_name']=='UCL Australia']

ucl_papers_without_geo = ucl_papers.iloc[:,:4]
ucl_papers_without_geo['institute_name']='University College London'

ucl_papers_geo = pd.merge(ucl_papers_without_geo,ucl,left_on='institute_name',right_on='institute_name')

In [ ]:
grid_matched_clean = pd.concat([grid_matched.loc[grid_matched['institute_name']!='UCL Australia'].reset_index(drop=True),
                                ucl_papers_geo],axis=0)

grid_matched_clean.head()

In [ ]:
lads = gp.read_file('../data/external/lad_shape/Local_Authority_Districts_December_2018_Boundaries_GB_BFC.shp')

In [ ]:
grid_matched_clean.to_csv(f'../data/external/{today_str}_papers_institution_ucl_cleaned.csv',compression='zip')

## Merge

In [ ]:
#This is to create the geodf
grid_df = gp.GeoDataFrame(
    grid_matched_clean, geometry=[Point(x, y) for x, y in zip(grid_matched_clean['institute_lon'], grid_matched_clean['institute_lat'])])

In [ ]:
#Reproject
lads = lads.to_crs({'init':'epsg:4326'})

In [ ]:
sp_joined = gp.sjoin(lads,grid_df,how='left',op='contains')

In [ ]:
papers_grouped = pd.concat([sp_joined.groupby('article_id')[var].apply(lambda x: list(x)) for var in ['lad18cd','lad18nm']],axis=1)

In [ ]:
papers_grouped.to_json(f'../data/processed/{today_str}_arxiv_lads.json',orient='index')